In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook


Bad key "text.kerning_factor" on line 4 in
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
Invitations=pd.read_csv(r'C:\Users\Andrea\Desktop\My_LinkedIn\Basic_Information_Files\Invitations.csv')
Invitations.tail()

,From,To,Sent At,Message,Direction
577,Aldo Javier Martinez Hutter,Marcela Huerfano,"12/5/19, 1:47 PM",NaN,INCOMING
578,Lina Marcela Escudero Góngora,Marcela Huerfano,"12/5/19, 1:43 PM",NaN,INCOMING
579,Edier Pacheco,Marcela Huerfano,"12/5/19, 11:42 AM",NaN,INCOMING
580,Andres Maldonado,Marcela Huerfano,"12/4/19, 5:51 PM",NaN,INCOMING
581,Absolute Latam,Marcela Huerfano,"12/4/19, 3:05 PM",NaN,INCOMING


In [3]:
Invitations.dtypes

From         object
To           object
Sent At      object
Message      object
Direction    object
dtype: object

In [4]:
rule1 = r'(?P<Month>\d{1,2})[/](?P<Day>\d{1,2})[/](?P<Year>\d{2})'
rule2 = r'(?P<Date>\d{1,2}[/]\d{1,2}[/]\d{2})'

extract_df = Invitations['Sent At'].str.lower().str.extract(rule1)
extract_df['Date']=Invitations['Sent At'].str.lower().str.extract(rule2)
extract_df.head()

,Month,Day,Year,Date
0,12,12,20,12/12/20
1,12,11,20,12/11/20
2,12,11,20,12/11/20
3,12,11,20,12/11/20
4,12,9,20,12/9/20


In [5]:
Invitations=pd.concat([Invitations, extract_df], axis=1)
Invitations.tail()

,From,To,Sent At,Message,Direction,Month,Day,Year,Date
577,Aldo Javier Martinez Hutter,Marcela Huerfano,"12/5/19, 1:47 PM",NaN,INCOMING,12,5,19,12/5/19
578,Lina Marcela Escudero Góngora,Marcela Huerfano,"12/5/19, 1:43 PM",NaN,INCOMING,12,5,19,12/5/19
579,Edier Pacheco,Marcela Huerfano,"12/5/19, 11:42 AM",NaN,INCOMING,12,5,19,12/5/19
580,Andres Maldonado,Marcela Huerfano,"12/4/19, 5:51 PM",NaN,INCOMING,12,4,19,12/4/19
581,Absolute Latam,Marcela Huerfano,"12/4/19, 3:05 PM",NaN,INCOMING,12,4,19,12/4/19


In [6]:
Invitations.dtypes

From         object
To           object
Sent At      object
Message      object
Direction    object
Month        object
Day          object
Year         object
Date         object
dtype: object

In [7]:
Invitations['Date']=pd.to_datetime(Invitations['Date'])